In [1]:
import pandas as pd

In [15]:
# Sample DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 30, 22, 28, 35],
}

df = pd.DataFrame(data)
print(df)

      Name  Age
0    Alice   25
1      Bob   30
2  Charlie   22
3    David   28
4      Eva   35


In [16]:
df2 = df.drop(2).drop(0)
df2

,Name,Age
1,Bob,30
3,David,28
4,Eva,35


In [17]:
idx = df2.index.get_loc(3)
print(idx)
df2.iloc[idx]

1


Name    David
Age        28
Name: 3, dtype: object

In [19]:
#next row from a specified idx
df2.iloc[idx+1]

Name    Eva
Age      35
Name: 4, dtype: object

In [21]:
df2.iloc[idx+1].name

4

In [33]:
s = "arg1 As Integer, arg2, ByRef arg2, Optional ByVal arg3, Optional ByVal arg4 As Integer"

lst_args = s.split(", ")
lst_args

['arg1 As Integer',
 'arg2',
 'ByRef arg2',
 'Optional ByVal arg3',
 'Optional ByVal arg4 As Integer']

In [34]:
s2.split(" As ")

['Optional ByVal arg1']

In [49]:
def ParseArg(arg):
    HasType = False
    lst = arg.split(" As ")
    if len(lst) > 1:
        HasType = True
        lst = lst[0].split(" ") + [lst[1]]
    else:
        lst = arg.split(" ")
    return lst, HasType

In [57]:
for arg in lst_args:
    parsed_arg, HasType = ParseArg(arg)
    print('\n', parsed_arg, HasType)
    
    IsOptional, HasBy, arg_by = False, False, "ByRef"
    if parsed_arg[0] == "Optional":
        IsOptional = True
        parsed_arg = parsed_arg[1:]
    if parsed_arg[0] in ["ByRef", "ByVal"]: 
        HasBy = True
        arg_by = parsed_arg[0]
        parsed_arg = parsed_arg[1:]
    
    arg_name = parsed_arg[0]
    arg_type = "Variant"
    if HasType: arg_type = parsed_arg[-1]
    
    arg_code_plan = "|".join([arg_name, arg_type, arg_by])
    if IsOptional: arg_code_plan = arg_code_plan + "|Optional"
        
    #print(arg, "   ", arg_name, arg_type, arg_by, IsOptional)
    print(arg_code_plan)



 ['arg1', 'Integer'] True
arg1|Integer|ByRef

 ['arg2'] False
arg2|Variant|ByRef

 ['ByRef', 'arg2'] False
arg2|Variant|ByRef

 ['Optional', 'ByVal', 'arg3'] False
arg3|Variant|ByVal|Optional

 ['Optional', 'ByVal', 'arg4', 'Integer'] True
arg4|Integer|ByVal|Optional
